# Calibration Model 03: Generátor syntetických dat (Synthetic Data Generator)

### Počet kalibrovaných parametrů: 3

In [1]:
# Instalace potřebných knihoven
#%pip install pandas
#%pip install numpy
#%pip install seaborn matplotlib
#%pip install pymc
#%pip install arviz
#%pip install ipywidgets
#%pip install jupyterlab_widgets
#%pip install pytensor
#%pip install ipywidgets jupyterlab_widgets

In [2]:
# Import potřebných knihoven
import pandas as pd
import numpy as np

import pymc as pm
import arviz as az
import pytensor.tensor as pt

import seaborn as sns
import matplotlib.pyplot as plt

### Vstupní data

In [3]:
### Načtení z formátu netCDF

# Soubor je načten a přiřazen do proměnné ‚trace‘
other_path = '../../data/05_Calibration/posterior_trace_three.nc'
trace = az.from_netcdf(other_path)

In [4]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v_load,0.494,0.317,0.173,0.935,0.153,0.001,6.0,100.0,1.73
v_unload,0.661,0.487,0.200,1.408,0.230,0.003,6.0,109.0,1.73
accel,0.205,0.007,0.200,0.217,0.001,0.001,42.0,131.0,1.06
sigma,3.405,0.095,3.230,3.584,0.001,0.001,9956.0,7339.0,1.00


### Definice a nastavení parametrů robotického systému

In [5]:
# SPECIFIKACE TECHNOLOGICKÉHO PROCESU ZDĚNÍ

time_suck_on = 1.5        # s, doba přisátí zdicího prvku
time_suck_off = 0.5       # s, doba odsátí (uvolnění) prvku
time_verif_refer = 3.5    # s, doba pohybu z místa ověření do referenčního bodu
time_refer_pallete = 2.1  # s, doba pohybu z referenčního bodu k paletě

time_pallete_verif = 9.8  # s, doba pohybu od palety k referenčnímu bodu

### Definice funkce pro výpočet celkové doby pracovního cyklu

In [6]:
def simulate_time(dist, v_load, v_unload, accel):
    """
    Funkce pro výpočet celkové doby pracovního cyklu robotického zdění.

    Návratová hodnota:
    total_time (float): celková doba pracovního cyklu [s]
    """

    # výpočet času potřebného k dosažení maximální rychlosti
    time_to_max_load_speed = v_load / accel
    dist_to_max_load_speed = (1/2) * accel * time_to_max_load_speed  # uražená dráha při akceleraci

    # výpočet času potřebného k dosažení 0 rychlosti
    time_to_max_unload_speed = v_unload / accel
    dist_to_max_unload_speed = (1/2) * accel * time_to_max_unload_speed  # uražená dráha při deakceleraci


    # pevné technologické časy (manipulace a přesuny mezi pevnými body)
    total_time = time_suck_on + time_pallete_verif
    total_time += time_suck_off + time_suck_on + time_verif_refer

    # pohyb s naloženým prvkem (převod mm → m)
    total_time += (dist - dist_to_max_load_speed) / 1_000 / v_load

    # manipulace v cílové poloze a návrat
    total_time += time_suck_off + time_refer_pallete

    # pohyb bez zátěže (zpětný pohyb)
    total_time += (dist - dist_to_max_unload_speed) / 1_000 / v_unload

    # započtení akceleračních časů
    total_time += time_to_max_load_speed + time_to_max_unload_speed
    return total_time

### Vymezení pracovního rozsahu pro generování dat

In [7]:
# počet scénářů simulace
number_simulation = 1000

# nastavení limitních hodnot rozsahu
dist_min = 64
dist_max = 5_026

# Vygeneruje 1000 náhodných hodnot z rovnoměrného rozdělení
dist_range = np.random.uniform(dist_min, dist_max, number_simulation)

### Empirická bootstrap distribuce zdržení (stochastické vlivy)

In [8]:
### Načtení hybridního modelu

# Soubor je načten a přiřazen do proměnné ‚stochastic_delay‘
other_path = '../../data/04_HybridModel/hybrid_model.csv'
stochastic_delay = pd.read_csv(other_path, header=0)

In [9]:
delay_samples = stochastic_delay["stochastic_delay"].values
delay_samples

array([0, 0, 0, ..., 0, 0, 0])

### Generátor synteticých dat

In [10]:
# =====================================================
# 1️ EXTRAKCE POSTERIOR VZORKŮ
# =====================================================

# Sloučení chain + draw do jedné dimenze
posterior = trace.posterior.stack(sample=("chain", "draw"))

# Extrakce parametrů
v_load_samples = posterior["v_load"].values
v_unload_samples = posterior["v_unload"].values
accel_samples = posterior["accel"].values
sigma_samples = posterior["sigma"].values

n_samples = len(v_load_samples)

In [ ]:
# =====================================================
# 2️ MONTE CARLO SIMULACE
# =====================================================

simulated = []
for d in dist_range:

    T_samples_for_dist = []

    for i in range(n_samples):

        T_det = simulate_time(
            dist=d,
            v_load=v_load_samples[i],
            v_unload=v_unload_samples[i],
            accel=accel_samples[i]
        )
        # bootstrap náhodná realizace
        delay = np.random.choice(delay_samples)

        # šum (noise) s normálním rozdělením
        #delay = np.random.normal(0, sigma_samples[i], size=np.shape(T_det))

        T_sim = T_det + delay

        T_samples_for_dist.append(T_sim)

    simulated.append(T_samples_for_dist)

simulated_data = np.array(simulated)

In [12]:
simulated_data.shape

(1000, 12000)

Simulace obsahuje:
* 1 000 hodnot scénářů
* 12 000 Monte Carlo realizací pro každý scénář

### Základní vyhodnocení (pro každý scenář)

In [13]:
# =====================================================
# 3️ STATISTICKÉ VYHODNOCENÍ
# =====================================================

# Střední hodnota
T_mean = simulated_data.mean(axis=1)

# Směrodatná odchylka
T_std = simulated_data.std(axis=1)

# 95% interval spolehlivosti
T_lower = np.percentile(simulated_data, 2.5, axis=1)
T_upper = np.percentile(simulated_data, 97.5, axis=1)

# Medián
T_median = np.median(simulated_data, axis=1)

In [14]:
# =====================================================
# 4️ VÝSTUPNÍ SHRNUTÍ
# =====================================================

print("Průměr:", T_mean)
print("95% interval:", T_lower, "-", T_upper)
print("Směrodatná odchylka:", T_std)
print("Medián:", T_median)

Průměr: [60.29669025 59.70006498 45.67205388 54.30203507 59.79100616 58.49844472
 60.4764698  47.15840352 41.96663521 43.76807784 53.34357952 34.19246546
 39.94774164 49.47121072 52.00272411 38.2844485  35.0581243  36.40034937
 41.19935489 46.21052117 37.08415168 32.80261007 50.78445677 46.30231958
 41.03545391 47.53857069 55.08243619 45.51789055 43.47605916 42.98373828
 46.51727083 60.97610714 49.15915427 47.977499   36.55670463 48.07380504
 45.1968468  33.58168621 51.2371625  33.63180495 52.85276824 55.45062277
 33.31804602 49.57739937 40.70565557 34.28704975 47.42983045 55.15776476
 52.14640745 54.88409176 49.18305954 51.48460276 54.80911206 41.98303806
 31.89291982 60.46714301 51.22550637 47.94683302 46.54873074 47.62574881
 53.13891848 43.5576049  49.03333044 60.45126804 57.95582635 32.67979582
 39.61910579 50.79148362 49.66599507 56.0554595  37.4134933  44.6646939
 55.33384796 36.36724215 52.69982341 52.52288498 49.37916962 52.93998665
 42.42857222 42.95226478 31.54496051 32.9744

### Globální vyhodnocení (přes všechny scénáře)

In [15]:
global_distribution = simulated_data.flatten()

global_mean = global_distribution.mean()
global_std = global_distribution.std()

global_lower = np.percentile(global_distribution, 2.5)
global_upper = np.percentile(global_distribution, 97.5)

global_median = np.median(global_distribution)

In [16]:
print("Globální průměr:", global_mean)
print("Globální 95% interval:", global_lower, "-", global_upper)
print("Globální Směrodatná odchylka:", global_std)
print("Globální medián:", global_median)

Globální průměr: 45.571007931480686
Globální 95% interval: 25.784947227928875 - 56.10976316978112
Globální Směrodatná odchylka: 69.10891065716714
Globální medián: 41.48750716782864


In [ ]:
results_df = pd.DataFrame({
    "dist": dist_range,
    "global_mean": global_mean,
    "global_CI_lower": global_lower,
    "global_CI_upper": global_upper,
    "global_std": global_std,
    "global_median": global_median,
    "T_mean": T_mean,
    "T_CI_lower": T_lower,
    "T_CI_upper": T_upper,
    "T_std": T_std,
    "T_median": T_median,

})

In [20]:
simulated_data

array([[54.40984676, 54.72769767, 54.77214331, ..., 54.13492068,
        54.67922412, 54.66243582],
       [53.70685752, 54.02666423, 54.06669212, ..., 53.43394478,
        53.94972416, 53.95806859],
       [41.17582898, 41.53049846, 41.49177856, ..., 40.93880479,
        40.94613275, 41.40247703],
       ...,
       [48.92560904, 49.25871774, 49.2686992 , ..., 48.66638968,
        48.98816802, 49.16744803],
       [40.97294835, 41.32818227, 41.28818743, ..., 40.7365052 ,
        40.7356012 , 41.19919872],
       [47.40576496, 47.74310204, 47.74353246, ..., 47.15089839,
        47.41100846, 47.64462478]])

### Generování datasetu syntetických dat

In [21]:
n_dist = simulated_data.shape[0]
n_mc = simulated_data.shape[1]

# zopakujeme každé dist 12000x
dist_long = np.repeat(dist_range, n_mc)

# rozbalíme časy
total_time_long = simulated_data.flatten()

# vytvoříme dataframe
synthetic_df = pd.DataFrame({
    "dist": dist_long,
    "total_time": total_time_long
})

In [22]:
synthetic_df.shape

(12000000, 2)

In [24]:
# náhodně podvzorkování
synthetic_df = synthetic_df.sample(500_000, random_state=122)

In [25]:
synthetic_df.head()

,dist,total_time
8875038,4549.348775,54.614786
9799933,3621.206464,48.164391
3368641,4758.314102,55.906718
10520965,3065.497002,47.067706
4406391,147.877841,26.301611


### Export datové sady do formátu CSV

In [33]:
synthetic_df.to_csv("../../data/05_Calibration/synthetic_dataset.csv", index=False)
results_df.to_csv("../../data/05_Calibration/results_synthetic_dataset.csv", index=False)

### Autor / Organizace / Datum

Vjačeslav Usmanov, ČVUT v Praze, Fakulta stavební

###### Přehled změn


|  Datum (YYYY-MM-DD) |  Verze | Autor změny  |  Popis změny |
|---|---|---|---|
| 2026-01-31 | 1.1 | Vjačeslav Usmanov| added CM_03_SyntGenerator.ipynb |
| 2026-02-18 | 1.2 | Vjačeslav Usmanov| changed CM_03_SyntGenerator.ipynb |